# IMPORT LIBRARY

In [47]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate   
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# Load PDF Package Langchain

In [48]:
# Load PDF
local_path = "visimisimandiri.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

PDF loaded successfully: visimisimandiri.pdf


Split Text Menjadi Potongan Dokumen untuk dijadikan Vektor

In [49]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Text split into {len(chunks)} chunks")

Text split into 5 chunks


# Initialized Vector Database With Chroma DB

In [50]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


# Setting LLM DAN Retrieval Engine

In [51]:
local_model = "llama3.2" 
llm = ChatOllama(model=local_model)

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# Create Template Chain (Context - Question - Answer)

In [52]:
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

# Prompting

In [53]:
chat_with_pdf("Apa Visi Misi dan Tujuan PT. Bank Mandiri?")

Visi Bank Mandiri pada periode 2020-2024 adalah "Menjadi Partner Finansial Pilihan Utama Anda". Dalam visi ini, bank berkomitmen untuk membangun hubungan jangka panjang dengan nasabah bisnis maupun perseorangan berdasarkan kepercayaan baik.

In [54]:
chat_with_pdf("Jelaskan Visi Bank Mandiri dalam Bahasa Indonesia")

Visi Bank Mandiri adalah "Spirit Memakmurkan Negeri". Dalam konteks ini, visi tersebut berarti Bank Mandiri ingin menjadi partner finansial yang paling dipercayai oleh masyarakat dan bisnis, dengan tujuan untuk membangun hubungan jangka panjang dengan nasabah, mendukung pertumbuhan ekonomi Indonesia, dan meningkatkan kinerja dan sumber daya manusia.

In [55]:
import onnxruntime as ort

# Buat session dengan provider GPU
providers = ort.get_available_providers()
if 'CUDAExecutionProvider' in providers:
    print("onnxruntime menggunakan GPU")
else:
    print("onnxruntime tidak menggunakan GPU")

onnxruntime tidak menggunakan GPU
